## Trabalho Prático - Bruno Silva - Grupo 7
### Unsupervised Learning: K-Means

O Dataset escolhido contem informações socioeconômicas, e de saúde, de 167 países, com o propósito de agrupá-los com base nessas informações.
O objetivo é perceber se o algoritmo é capaz de distinguir entre países desenvolvidos e em desenvolvimento.

Dicionário de Dados:
- country: Name of the country
- child_mort: Death of children under 5 years of age per 1000 live births
- exports: Exports of goods and services per capita. Given as %age of the GDP per capita
- health: Total health spending per capita. Given as %age of GDP per capita
- imports: Imports of goods and services per capita. Given as %age of the GDP per capita
- Income: Net income per person
- Inflation: The measurement of the annual growth rate of the Total GDP (Crescimento anual PIB)
- life_expec: The average number of years a new born child would live if the current mortality patterns are to remain the same
- total_fer: The number of children that would be born to each woman if the current age-fertility rates remain the same.
- gdpp: The GDP per capita. Calculated as the Total GDP divided by the total population. ( PIB )  

Link para o Dataset: [Unsupervised Learning on Country Data](https://www.kaggle.com/datasets/rohan0301/unsupervised-learning-on-country-data/data)


### Importar livrarias necessárias

In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objs as go
import plotly.figure_factory as ff

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import warnings

warnings.filterwarnings("ignore")

### Carregar o dataset

In [2]:
dados = pd.read_csv("Country-data.csv")
nomePaises = dados["country"].tolist()
dados.head()

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553
1,Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460
3,Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200


### Divisão dos Dados entre Categóricos e Numéricos
Todos os dados do conjunto de dados são numéricos, à exceção dos nomes dos países.  
Seria possível converter dados categóricos em numéricos (por exemplo, "bom/razoável/mau" -> "3/2/1") de modo a torná-los utilizáveis.

In [3]:
dadosNum = list(dados.columns)
dadosNum.remove("country")
dadosCate = ["country"]
print("Dados Categoricos :", dadosCate)
print("Dados Numéricos :", dadosNum)

Dados Categoricos : ['country']
Dados Numéricos : ['child_mort', 'exports', 'health', 'imports', 'income', 'inflation', 'life_expec', 'total_fer', 'gdpp']


### Verificar se não existe países repetidos

In [4]:
# len(dados['country'].unique()) -> verificar quantos valores unicos existem
# len(dados) -> Quantas linhas tem o dataframe
len(dados["country"].unique()) == len(dados)

True

### Verificar se os dados seguem uma distribuição normal.  


In [5]:
# Criar janela para agrupar os subplots
fig = sp.make_subplots(rows=3, cols=3, subplot_titles=dadosNum)

# Adicionar subplots com distribuições
# Necessário tanto o nome das coluna (coluna) como o indice da mesma na lista (i)
for i, coluna in enumerate(dadosNum):
    plot = go.Histogram(x=dados[coluna])
    fig.add_trace(plot, row=(i // 3) + 1, col=(i % 3) + 1)

# Layout da janela com os subplots
fig.update_layout(
    height=600,
    width=1000,
    title_text="Distribuição dos Dados Numéricos",
)

# Mostrar o gráfico
fig.show()

### Análise Individual de Cada Variável
Vamos analisar os quatro países com os valores mais altos e mais baixos para cada variável, assim como aqueles que se encontram na média.  
Embora não seja um passo essencial, é sempre importante e interessante analisar as variáveis, quando possível.

In [6]:
def plot_subplots(dados, column, title):
    data_sorted = dados.sort_values(by=column, ascending=False)

    # Extracting data for each subplot
    high_data = data_sorted.iloc[:5]
    medium_data = data_sorted.iloc[81:86]
    low_data = data_sorted.iloc[161:166]

    # Creating subplots
    fig = go.Figure()

    # Subplot 1
    fig.add_trace(
        go.Bar(
            x=high_data["country"],
            y=high_data[column],
            marker=dict(color="blue"),
            name=f"Alta {column}",
        )
    )

    # Subplot 2
    fig.add_trace(
        go.Bar(
            x=medium_data["country"],
            y=medium_data[column],
            marker=dict(color="red"),
            name=f"Media {column}",
        )
    )

    # Subplot 3
    fig.add_trace(
        go.Bar(
            x=low_data["country"],
            y=low_data[column],
            marker=dict(color="green"),
            name=f"Baixa {column}",
        )
    )

    # Updating layout
    fig.update_layout(
        barmode="group",
        title=title,
        xaxis=dict(tickangle=45),
        yaxis=dict(title=column),
        showlegend=True,
        width=600,
        height=400,
    )

    # Show the plot
    fig.show()


# Imprimir plots de cada coluna
plot_subplots(
    dados,
    "child_mort",
    "Morte de crianças com menos de 5 anos<br>de idade por cada 1000",
)
plot_subplots(
    dados,
    "exports",
    "Exportações de bens e serviços per capita<br>Dado em percentagem do PIB per capita",
)
plot_subplots(
    dados,
    "health",
    "Despesas totais de saúde per capita<br>Dado em percentagem do PIB per capita",
)
plot_subplots(dados,
    "imports",
    "Importações de bens e serviços per capita<br>Em percentagem do PIB per capita",
)
plot_subplots(dados, "income", "Rendimento líquido anual por pessoa")
plot_subplots(dados, "inflation", "Taxa de inflação")
plot_subplots(dados, "life_expec", "Esperança média de vida")
plot_subplots(dados, "total_fer", "Numero de filhos por mulher")
plot_subplots(dados, "gdpp", "O PIB per capita")

Dos gráficos acima, é possível perceber alguns padrões, como, por exemplo, os países africanos que frequentemente apresentam indicadores desfavoráveis, como baixo rendimento, baixa esperança média de vida ou alta inflação. Por outro lado, temos países com reconhecida qualidade de vida, como Suíça ou Noruega, que exibem indicadores excelentes.  

Um dado interessante é que os Estados Unidos lideram as despesas em saúde per capita, no entanto, não lideram os indicadores de saúde, como mortalidade infantil ou esperança média de vida.


### Matriz de Correlação
É crucial examinar a correlação entre variáveis, uma vez que altas correlações podem distorcer os resultados.

In [7]:
heatData = dados.drop("country", axis=1)
correlation_matrix = heatData.corr()

# Arredondar os valores da matriz de correlação para 2 casas decimais
correlation_matrix = correlation_matrix.round(2)

fig = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=list(correlation_matrix.columns),
    y=list(correlation_matrix.index),
    colorscale="Viridis",
    colorbar=dict(title="Correlation Coefficient", tickvals=[-1, -0.5, 0, 0.5, 1]),
)

fig.update_layout(
    height=600,  # Ajuste a altura conforme necessário
    width=800,
    title="Matriz de Correlação",
    xaxis=dict(tickangle=-45),
    yaxis=dict(tickangle=45),
)

# Mostrar o gráfico
fig.show()

### Observações na Matriz de Correlação
- Os rendimentos (income) estão claramente relacionados com o PIB per capita (gdpp).
- A mortalidade infantil, taxa de fertilidade e esperança média de vida também apresentam correlação.
- As exportações também mostram relação com as importações.

Altas correlações podem causar problemas com colinearidade e redundância de informação, o que pode ser prejudicial para a análise de dados. Para lidar com elevadas correlações, vamos aplicar uma análise PCA e utilizar os dados resultantes numa análise K-Means.

### Normalização dos Dados
Para trabalhar com os dados, é crucial normalizá-los; caso contrário, algumas variáveis podem exercer pesos diferentes nos resultados.  
O algoritmo PCA também pressupõe que os dados estejam normalizados.


In [8]:
# Criar nova dataframe sem a variavel "country"
dadosNorm = dados.copy()
dadosNorm = dadosNorm.drop("country", axis=1)

# Normalizar os dados
scaler = MinMaxScaler()
dadosNorm = pd.DataFrame(scaler.fit_transform(dadosNorm))
dadosNorm.head()

,0,1,2,3,4,5,6,7,8
0,0.426485,0.049482,0.358608,0.257765,0.008047,0.126144,0.475345,0.736593,0.003073
1,0.068160,0.139531,0.294593,0.279037,0.074933,0.080399,0.871795,0.078864,0.036833
2,0.120253,0.191559,0.146675,0.180149,0.098809,0.187691,0.875740,0.274448,0.040365
3,0.566699,0.311125,0.064636,0.246266,0.042535,0.245911,0.552268,0.790221,0.031488
4,0.037488,0.227079,0.262275,0.338255,0.148652,0.052213,0.881657,0.154574,0.114242


### Análise de Componentes Principais (PCA):

O PCA é amplamente utilizado em Unsupervised Learning, pois auxilia na redução da dimensão dos dados, sendo útil para conjuntos de dados extensos.  Além disso, ajuda a eliminar redundâncias nas medições e a reduzir o "ruído" que pode estar presente nos dados. Dessa forma, conseguimos aprimorar a qualidade dos dados e, ao mesmo tempo, aumentar o desempenho do modelo.


In [9]:
pca = PCA()
dadosPCA = pd.DataFrame(pca.fit_transform(dadosNorm))

# Extrair a variância cumulativa
variancia_cumulativa = np.cumsum(pca.explained_variance_ratio_)

# Criar o gráfico
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=list(range(1, 20)),
        y=variancia_cumulativa,
        mode="lines+markers",
        name="Variância Cumulativa",
    )
)

fig.update_layout(
    width=1000,
    xaxis_title="Número de Componentes Principais",
    yaxis_title="Variância Cumulativa",
    title="Análise de Variância Cumulativa do PCA",
)

# Mostrar o gráfico
fig.show()

Ao analisar os dados gerados pelo PCA, observamos que conseguimos preservar 97% da informação utilizando apenas 6 componentes, permitindo-nos descartar os 3 restantes.  
A redução na dimensão dos nossos dados é evidente, passando de 1503 para 1002 com a aplicação do PCA. Isso representa uma redução de aproximadamente 33%. Em conjuntos de dados extensos, essa redução pode traduzir-se numa redução valiosa de tempo e recursos.


In [10]:
dadosPCA = dadosPCA.drop(columns=[6, 7, 8])
print("Tamanho do dataset com PCA:", dadosPCA.size)
print("Tamanho do dataset sem PCA:", dadosNorm.size)

dadosPCA.insert(0, "country", nomePaises)
dadosNorm.insert(0, "country", nomePaises)


Tamanho do dataset com PCA: 1002
Tamanho do dataset sem PCA: 1503


In [11]:
dadosPCA.head()

,country,0,1,2,3,4,5
0,Afghanistan,-0.599078,0.095490,0.157554,-0.024333,-0.045618,-0.046532
1,Albania,0.158474,-0.212092,-0.064189,-0.061247,0.014191,-0.010246
2,Algeria,0.003686,-0.135867,-0.134182,0.133574,-0.091150,0.025988
3,Angola,-0.650235,0.275975,-0.142672,0.156018,-0.081997,0.032170
4,Antigua and Barbuda,0.200711,-0.064662,-0.100715,-0.037902,-0.035799,-0.055817


In [12]:
dadosNorm.head()

,country,0,1,2,3,4,5,6,7,8
0,Afghanistan,0.426485,0.049482,0.358608,0.257765,0.008047,0.126144,0.475345,0.736593,0.003073
1,Albania,0.068160,0.139531,0.294593,0.279037,0.074933,0.080399,0.871795,0.078864,0.036833
2,Algeria,0.120253,0.191559,0.146675,0.180149,0.098809,0.187691,0.875740,0.274448,0.040365
3,Angola,0.566699,0.311125,0.064636,0.246266,0.042535,0.245911,0.552268,0.790221,0.031488
4,Antigua and Barbuda,0.037488,0.227079,0.262275,0.338255,0.148652,0.052213,0.881657,0.154574,0.114242


### Aplicar o K-Means
Vamos agora aplicar o K-Means aos dados provenientes do PCA e também aos dados sem tratamento PCA para comparar resultados

In [13]:
kMeans_PCA = dadosPCA.drop(columns=["country"]).values
kMeans_NoPCA = dadosNorm.drop(columns=["country"]).values

### Descobrir valor para K
Para descobrir o valor ideal de clusters vamos usar a conhecida técnica do "cotovelo"

In [14]:
sse_PCA = {}
kmax = 10

# Elbow Method
fig = sp.make_subplots(
    rows=1, cols=1, subplot_titles=("Método do Cotovelo"), column_widths=[1]
)

# Elbow Method
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(kMeans_PCA)
    sse_PCA[
        k
    ] = (
        kmeans.inertia_
    )  # Inertia: Sum of distances of samples to their closest cluster center

elbow_trace = go.Scatter(
    x=list(sse_PCA.keys()), y=list(sse_PCA.values()), mode="lines+markers"
)
fig.add_trace(elbow_trace, row=1, col=1)
fig.update_xaxes(title_text="k: Número de clusters", row=1, col=1)
fig.update_yaxes(title_text="Soma do erro quadrático", row=1, col=1)

fig.update_layout(
    title_text="Método de avaliação de clusters para dados com PCA",
    showlegend=False,
    width=700,
)
fig.show()

Podemos verificar pelo gráfico que o numero ideal de clusters é 3

In [15]:
sse_NoPCA = {}
kmax = 10

# Elbow Method
fig = sp.make_subplots(
    rows=1, cols=1, subplot_titles=("Método do Cotovelo"), column_widths=[1]
)

# Elbow Method
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(kMeans_PCA)
    sse_PCA[
        k
    ] = (
        kmeans.inertia_
    )  # Inertia: Sum of distances of samples to their closest cluster center

elbow_trace = go.Scatter(
    x=list(sse_PCA.keys()), y=list(sse_PCA.values()), mode="lines+markers"
)
fig.add_trace(elbow_trace, row=1, col=1)
fig.update_xaxes(title_text="k: Número de clusters", row=1, col=1)
fig.update_yaxes(title_text="Soma do erro quadrático", row=1, col=1)

fig.update_layout(
    title_text="Método de avaliação de clusters para dados com PCA",
    showlegend=False,
    width=700,
)
fig.show()

Com base nos gráficos acima, o numero de clusters ideal será de 3 tanto para os dados tratados com PCA como para os dados não tratados.

### Aplicar K-means com K=3 (com PCA)


In [16]:
# Set a random seed for reproducibility
np.random.seed(42)

# Assuming m2 is your data
modelPCA = KMeans(n_clusters=3, max_iter=1000)
modelPCA.fit(kMeans_PCA)
cluster = modelPCA.cluster_centers_
centroids = np.array(cluster)
labels = modelPCA.labels_

# Assuming pca_df is your PCA DataFrame
pca = PCA(n_components=3)
pca_result = pca.fit_transform(kMeans_PCA)
PCA_df = pd.DataFrame(
    data=pca_result, columns=["1th Component", "2st Component", "3nd Component"]
)

# Add cluster labels to the DataFrame
PCA_df["Class"] = labels

# Create a 3D scatter plot using Plotly
fig = go.Figure()


# Scatter plot for data points colored by cluster
for cluster_label in np.unique(labels):
    cluster_indices = labels == cluster_label
    fig.add_trace(
        go.Scatter3d(
            x=PCA_df.loc[cluster_indices, "1th Component"],
            y=PCA_df.loc[cluster_indices, "2st Component"],
            z=PCA_df.loc[cluster_indices, "3nd Component"],
            mode="markers",
            marker=dict(size=8),
            name=f"Cluster {cluster_label}",
        )
    )

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title="1th Component",
        yaxis_title="2st Component",
        zaxis_title="3nd Component",
    ),
    title="K-Means com PCA",
    showlegend=True,
)

# Show the plot
fig.show()

Abaixo vamos usar um mapa choropleth para visualizar facilmente o agrupamento dos países.  
É também importante apresentar os resultados de uma forma que seja possível analisar com facilidade.

In [17]:
PCA_df.insert(0, column="Country", value=dados["country"])

PCA_df["Class"].loc[PCA_df["Class"] == 0] = "Cluster 1"
PCA_df["Class"].loc[PCA_df["Class"] == 1] = "Cluster 2"
PCA_df["Class"].loc[PCA_df["Class"] == 2] = "Cluster 3"

fig = px.choropleth(
    PCA_df[["Country", "Class"]],
    locationmode="country names",
    locations="Country",
    title="Agrupamento com dados PCA",
    color=PCA_df["Class"],
    color_discrete_map={
        "Cluster 1": "Blue",
        "Cluster 2": "Yellow",
        "Cluster 3": "Green",
    },
)
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(
    legend_title_text="Labels",
    legend_title_side="top",
    title_pad_l=260,
    title_y=0.86,
    width=1000,
    coloraxis_colorbar=dict(title="Assistance Level"),
)
fig.show(engine="kaleido")

Pelo mapa acima podemos verificar claramente que o algoritmo agrupou, embora com alguma imperfeição, países desenvolvidos, subdesenvolvidos, e países que embora não estão dentro do grupo de países mais ricos, ou com melhores condições, também não estão em tão má posição como os países subdesenvolvidos.
Verificamos com naturalidade que Africa é um continente onde prevalece países com piores condições.

### Aplicar K-Means com K=3 (sem PCA)

In [18]:
# Set a random seed for reproducibility
np.random.seed(42)

# Assuming m2 is your data
modelNoPCA = KMeans(n_clusters=3, max_iter=1000)
modelNoPCA.fit(kMeans_NoPCA)
cluster = modelNoPCA.cluster_centers_
centroids = np.array(cluster)
labels = modelNoPCA.labels_

# Assuming pca_df is your PCA DataFrame
NoPCA = PCA(n_components=3)
NoPCA_result = NoPCA.fit_transform(kMeans_NoPCA)
NoPCA_df = pd.DataFrame(
    data=NoPCA_result, columns=["1th Component", "2st Component", "3nd Component"]
)

# Add cluster labels to the DataFrame
NoPCA_df["Class"] = labels

# Create a 3D scatter plot using Plotly
fig = go.Figure()


# Scatter plot for data points colored by cluster
for cluster_label in np.unique(labels):
    cluster_indices = labels == cluster_label
    fig.add_trace(
        go.Scatter3d(
            x=NoPCA_df.loc[cluster_indices, "1th Component"],
            y=NoPCA_df.loc[cluster_indices, "2st Component"],
            z=NoPCA_df.loc[cluster_indices, "3nd Component"],
            mode="markers",
            marker=dict(size=8),
            name=f"Cluster {cluster_label}",
        )
    )

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title="1th Component",
        yaxis_title="2st Component",
        zaxis_title="3nd Component",
    ),
    title="K-Means sem PCA",
    showlegend=True,
)

# Show the plot
fig.show()

In [19]:
NoPCA_df.insert(0, column="Country", value=dados["country"])

NoPCA_df["Class"].loc[NoPCA_df["Class"] == 0] = "Cluster 1"
NoPCA_df["Class"].loc[NoPCA_df["Class"] == 1] = "Cluster 2"
NoPCA_df["Class"].loc[NoPCA_df["Class"] == 2] = "Cluster 3"

fig = px.choropleth(
    NoPCA_df[["Country", "Class"]],
    locationmode="country names",
    locations="Country",
    title="Agrupamento com dados PCA",
    color=NoPCA_df["Class"],
    color_discrete_map={
        "Cluster 1": "Blue",
        "Cluster 2": "Green",
        "Cluster 3": "Yellow",
    },
)
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(
    legend_title_text="Labels",
    legend_title_side="top",
    title_pad_l=260,
    title_y=0.86,
    width=1000,
    coloraxis_colorbar=dict(title="Assistance Level"),
)
fig.show(engine="kaleido")

Nos dados sem tratamento PCA obtemos os mesmo resultados, em muitos casos obtemos melhores resultados aplicando o PCA, mas nas piores das hipoteses, é pelo menos possivel reduzir a dimensão dos nossos dados para poupar tempo e recursos.